# Calculate and save the upward and downward mass fluxes for each 3D file

$M = \rho w $

In [1]:
import numpy as np
import xarray as xr
import glob
import os
from datetime import datetime

In [2]:
# today's date as a string

today = datetime.today().strftime('%Y-%m-%d')

In [3]:
# pre-defined fields

R = 287 # J/kg gas constant

simset = 'rcemip-small' # options: 'rcemip-small' or 'rcemip-large'

dirout = 'massflux'

cases = ['nz_32','nz_64','nz_128','nz_256']

# directory where data is located
fpath = 

# author, your name
author = 

# email, your email
email = 

In [4]:
fpath = fpath + simset

if simset == 'rcemip-small':
    dx = 780
    outstr1 = '_99km_300K_'
    nxny = '_128_'
    
else:
    
    cases = cases[:2]
    dx = 3_000
    outstr1 = '_1536km_300K_'
    nxny = '_512_'

In [5]:
# Loop through files and calculate one by one

for icase,case in enumerate(cases):
    
    try:
        os.mkdir(fpath + case + '/OUT_3D/' + dirout)
    except:
        print('directory already exists!')

    # Load background pressure file and convert to Pa from hPa
    p_background = xr.open_dataset(glob.glob(fpath + case +  
                                            '/OUT_3D/p/*.nc')[0]).p * 100

    # List files: need w, pressure, and temperature
    files_w = np.sort(glob.glob(fpath + case + '/OUT_3D/W/*.nc'))
    files_p = np.sort(glob.glob(fpath + case + '/OUT_3D/PP/*.nc'))
    files_t = np.sort(glob.glob(fpath + case + '/OUT_3D/TABS/*.nc'))

    # Loop through files
    for ifile in range(len(files_w)):

        tstr = files_w[ifile][-13:-3] # time string

        # Check to see if file exists & if it does, continue
        filename_out = (fpath + case+ '/OUT_3D/' + dirout + '/RCE_' + simset + outstr1 +
                case + nxny + dirout + '_' + tstr + '.nc')

        if os.path.exists(filename_out): 
            print('file exists! Skipping this one')
            continue

        # Open files
        w = xr.load_dataset(files_w[ifile]).W
        T = xr.load_dataset(files_t[ifile]).TABS
        p = xr.load_dataset(files_p[ifile]).PP + p_background

        # Calculate density = pressure/(R * T)
        rho = p/(R*T)
        del T
        del p

        # Calculate mass flux for each column
        M = rho * w
        del w
        del rho

        M.attrs['long_name'] = 'mass flux'
        M.attrs['units'] = 'kg/m2/s'

        # --------- Save to netcdf ---------
        ds_out = xr.Dataset(data_vars=dict(M=M),
                            attrs=dict(history='calculated on ' + today +
                                               ' by ' + author + email))
        ds_out.to_netcdf(filename_out)